# Fit models to human data

In [ ]:
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm

import sys
sys.path.append("../src")

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from Config.config import PATHS
from Classes.parameter_recovery import ParameterFit
from Classes.cognitive_model_agents import MODELS

### Load human data

In [ ]:
# Load data into a dataframe

two_player = False  # Set to True for 2-player UR experiment, False for multi-player IU experiment
if two_player:
	file_name = '2-player-UR.csv' # <= Data from 2-player UR experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_2P.json')
else:
	# file_name = '3-player-IU.csv ' # <= Data from 3-player IU experiment
	# file_name = '4-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '5-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '6-player-IU.csv' # <= Data from 3-player IU experiment
	file_name = '7-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '8-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '9-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '11-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = '12-player-IU.csv' # <= Data from 3-player IU experiment
	# file_name = 'multi-player.csv' # <= Data from multi-player IU experiment
	best_fits_file = PATHS['parameter_fit_results'] / Path('best_fit_MP.json')

data_folder = PATHS['human_data']
file = data_folder / Path(file_name)
print(f'Loading data from {file}...')

data = pd.read_csv(file)
columns = ['threshold', 'num_players', 'group', 'round', 'player', 'score', 'decision']
drop_columns = [col for col in data.columns if col not in columns]
data.drop(columns=drop_columns, inplace=True)
data.head(2)

### Select models

In [ ]:
# check_out_these = [model.name() for model in MODELS]
# check_out_these = ['AvailableSpace-M1', 'AvailableSpace-M2', 'AvailableSpace-M3']
# check_out_these = ['Attendance-M1', 'Attendance-M2', 'Attendance-M3'] 
# check_out_these = ['Payoff-M1', 'Payoff-M2', 'Payoff-M3']
# check_out_these = ['Fairness-M1', 'Fairness-M2', 'Fairness-M3']
# check_out_these = ['WSLS-M1', 'WSLS-M2', 'WSLS-M3']
# check_out_these = ['Payoff-M2', 'Fairness-M2', 'AvailableSpace-M2']
# check_out_these = ['Attendance-M2', 'MFP-M2', 'Payoff-M2']
# check_out_these = ['Payoff-M2', 'Fairness-M2']
# check_out_these = ['Fairness-M2']
# check_out_these = ['FRA']
# check_out_these = ['MFP-M3']
# check_out_these = ['FRA+Payoff+Attendance']
check_out_these = ['Priors-M3']
my_models = [model for model in MODELS if model.name() in check_out_these]

### Fit with scipy

In [ ]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Scipy.json',
    optimizer_type='scipy',
    hyperparameters=None,
    new_file=True
)

### Fit with Bayesian Optimizer

In [ ]:
hyperparameters = {
    'init_points':128,
    'n_iter':64
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json',
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=True
)

### Keep best fit from both optimizers

In [ ]:
# Load the best fit results from both optimizers
df_Scipy = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Scipy.json', lines=True)
df_Bayesian = pd.read_json(PATHS['parameter_fit_results'] / 'best_fit_Bayesian.json', lines=True)

df_list = []
best_optimizer = []
# Iterate through each model and compare the deviance from both optimizers
for model in my_models:
    best_fits = {'model_name': model.name()}
    # Filter the dataframes for the current model
    df1_Scipy = df_Scipy[df_Scipy['model_name'] == model.name()]
    df1_Bayesian = df_Bayesian[df_Bayesian['model_name'] == model.name()]
    # Iterate through each num_agents and threshold combination
    cols = [col for col in df1_Bayesian.columns if col != 'model_name']
    for col in cols:
        if not isinstance(df1_Scipy[col].values[0], dict):
            continue
        num_agents = df1_Scipy[col].values[0]['fixed_parameters']['num_agents']
        threshold = df1_Scipy[col].values[0]['fixed_parameters']['threshold']
        best_fits['num_agents'] = num_agents
        best_fits['threshold'] = threshold
        deviance_Scipy = df1_Scipy[col].values[0]['deviance']
        deviance_Bayesian = df1_Bayesian[col].values[0]['deviance']
        # Compare deviance from both optimizers
        if deviance_Scipy > deviance_Bayesian:
            best_optimizer.append('Scipy')
            best_fits['deviance'] = deviance_Scipy
            best_fits['AIC'] = df1_Scipy[col].values[0]['AIC']
            best_fits['free_parameters'] = df1_Scipy[col].values[0]['free_parameters']
        else:
            best_optimizer.append('Bayesian')
            best_fits['deviance'] = deviance_Bayesian
            best_fits['AIC'] = df1_Bayesian[col].values[0]['AIC']
            best_fits['free_parameters'] = df1_Bayesian[col].values[0]['free_parameters']
        best_fits['fixed_parameters'] = df1_Scipy[col].values[0]['fixed_parameters']
        df = pd.DataFrame().from_dict(best_fits, orient='index').T
        df_list.append(df)
df_best_fits = pd.concat(df_list, ignore_index=True)
print('Otimizer with best results:')
pd.Series(best_optimizer).value_counts(normalize=True).reset_index().rename(columns={'index': 'optimizer', 0: 'proportion'}).sort_values(by='proportion', ascending=False)


### Save to file

In [ ]:
new_file = False # Set to True if you want to create a new file, False to update the existing one

if new_file:
    df_best_fits.to_json(best_fits_file, orient='records', lines=True)
else:
    best_fit = pd.read_json(best_fits_file, lines=True)
    for model_name in df_best_fits['model_name'].unique():
        df_model = df_best_fits[df_best_fits['model_name'] == model_name]
        if model_name not in best_fit['model_name'].unique():
            print(f'Adding new model: {model_name}')
            best_fit = pd.concat([best_fit, df_model], ignore_index=True)
        else:
            list_fixed_parameters_new = df_model['fixed_parameters'].values
            for fixed_parameters_new in list_fixed_parameters_new:
                df_parameters = df_model[df_model['fixed_parameters'] == fixed_parameters_new]
                mask = (best_fit['model_name'] == model_name) & (best_fit['fixed_parameters'] == fixed_parameters_new)
                if mask.sum() == 0:
                    best_fit = pd.concat([best_fit, df_parameters], ignore_index=True)
                else:
                    # Update the existing entry with the new data
                    best_fit = best_fit.copy()                    
                    best_fit.loc[mask, df_parameters.columns] = df_parameters.values
                

    best_fit.to_json(best_fits_file, orient='records', lines=True)
    
print(f'Best fit results saved to {best_fits_file}')

---